## Calculating the distance to the Palomar 5 globular cluster

Author: Luke Bischoff<br>
Created: 11 DECEMBER 2020 11:22<br>
Updated: 11 DECEMBER 2020 11:25

##### Introduction
TTT
##### Contents
fff
##### Import list

In [1]:
import math
import glob
import copy
import numpy as np
import pandas as pd
import astroalign as aa
import matplotlib
import matplotlib.pyplot as plt
from gloess_plot import GLOESSpy
from scipy.optimize import curve_fit
from astropy import wcs
from astropy import units as u
from astropy.io import fits # used for FITS file management
from astropy.io import ascii
from astropy.time import Time
from astropy.stats import sigma_clipped_stats # used within star detection
from astropy.table import Table, join
from astropy.nddata import NDData
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
from astropy.coordinates import Angle
from astropy.visualization import simple_norm
from astropy.io.votable import parse_single_table
from astropy.modeling.fitting import LevMarLSQFitter
from photutils import aperture_photometry # used to perform photometry using annuli
from photutils import DAOStarFinder # used for the star finding algorithm
from photutils import CircularAperture, CircularAnnulus
from photutils import EPSFBuilder
from photutils.psf import extract_stars
from photutils.psf import DAOGroup
from photutils.psf import IterativelySubtractedPSFPhotometry
from photutils.background import MMMBackground
from astroquery.irsa_dust import IrsaDust
from matplotlib.colors import LogNorm

##### User defined functions*